In [1]:
import os
import numpy as np 
import matplotlib.pyplot as plt

from typing import List

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from imutils import paths

In [2]:
dataset = "data/cropped"

train_paths = sorted(list(paths.list_images(f"{dataset}/train")))
val_paths = sorted(list(paths.list_images(f"{dataset}/val")))
test_paths = sorted(list(paths.list_images(f"{dataset}/test")))

In [3]:
def create_dataset(paths: List[str]) -> np.ndarray:
    """
    Fonction permettant la création d'un jeu de données d'images à partir d'une liste
    de chemins d'accès à ces images.

    Parameters
    ----------
    paths: 
        Chemin d'accès des données

    Returns
    -------
    np.ndarray:
        Deux matrices contenant les images et leurs labels
    """
    data = []
    labels = []

    for path in paths:
        label = path.split(os.path.sep)[-2]
        image = load_img(path, target_size=(224, 224))
        image = img_to_array(image)
        data.append(image)
        labels.append(label)
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)

    # On encode les labels
    mlb = LabelBinarizer()
    labels = mlb.fit_transform(labels)
    
    return data, labels

In [4]:
X_train, y_train = create_dataset(train_paths)
X_val, y_val = create_dataset(val_paths)
X_test, y_test = create_dataset(test_paths)

In [5]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(335, 224, 224, 3) (335, 3)
(60, 224, 224, 3) (60, 3)
(60, 224, 224, 3) (60, 3)


In [10]:
model = Sequential([    
    Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(axis=-1),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(axis=-1),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(rate=0.3),
    
    Dense(3, activation='softmax')
])

In [11]:
lr = 0.001
epochs = 50

opt = Adam(learning_rate=lr, decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
# add ModelCheckpoint callback to record best weight only
checkpointer = ModelCheckpoint(monitor='val_loss',
                               filepath='saved_models_checkpoint/best_weights.h5',
                               verbose=1, save_best_only=True)

# add Earlystop callback
earlystopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10,
                              verbose=1, mode='auto', restore_best_weights=True)

# add ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=3, verbose=1)

In [15]:
from tensorflow.keras.backend import clear_session
clear_session()

history = model.fit(
    # aug.flow(X_train, y_train, batch_size=32),
    X_train, y_train, 
    epochs=epochs, 
    callbacks=[
        checkpointer, 
        earlystopping, 
        reduce_lr
    ],
    validation_data=(X_val, y_val)
)

Epoch 1/50


ResourceExhaustedError:  OOM when allocating tensor with shape[32,128,106,106] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/conv2d_7/Relu (defined at tmp/ipykernel_8978/961902480.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_3921]

Function call stack:
train_function


In [ ]:
# Evaluation du modèle
model.evaluate(X_test, y_test)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
pred = model.predict(X_test)

# Heatmap to see how images are classified
import seaborn as sns

conf = confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1))
plt.figure(figsize=(15, 15))
sns.heatmap(conf, cmap='mako', annot=True, fmt='d')

In [ ]:
# Save last checkpoint
model.save('model.h5')